In [ ]:
from pyspark.sql import SparkSession, functions as F

# 1. PySpark SQL의 API는 SparSession을 entry point로 접근할 수 있습니다.
spark: SparkSession = SparkSession.builder.getOrCreate()

# 2. SparkSession을 통하여 AWS S3에 Parquet 포맷으로 저장된 로그를
#    DataFrame으로 로드합니다.
economy_log: DataFrame = spark.read.parquet("s3a://logging-test-game/log/partition=economy")

# 3. DataFrame을 temp table로 등록해서 SQL로 쿼리합니다.
economy_log.registerTempTable("economy_log")
economy_table: DataFrame = spark.sql("""
      SELECT mid, sum(properties.amountDelta) AS amount
        FROM economy_log
    GROUP BY mid
""")

# 3-1. 또는 DataFrame API를 통하여 쿼리할 수도 있습니다.
economy_table: DataFrame = (
    economy_log
      .groupBy("mid")
      .agg(F.sum(F.col("properties.amountDelta")).alias("amount"))
      .select("mid", "amount")
)

# 4. 결과물을 S3로 다시 저장합니다.
economy_table.write.parquet("s3a://analytics-test-game/dw/f_economy")